In [ ]:


import numpy as np
from scipy.stats import t, chi2

class RegresionLinealSimple():
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
        self.n = len(x) # Tamaño de muestra
        self.x_bar = np.mean(x) # Media de x
        self.y_bar = np.mean(y) # Media de y
        self.ajustar_modelo()
        
        
    
    def ajustar_modelo(self):
        Sxy = np.sum( (self.x-self.x_bar) * (self.y-self.y_bar) )
        Sxx = np.sum( (self.x-self.x_bar) ** 2 )
    
        b1 = Sxy/Sxx
        b0 = self.y_bar - b1 * self.x_bar
        print("B0=", b0, ", B1=", b1 )
        
        self.Sxx = Sxx
        self.Sxy = Sxy
        
        self.b0 = b0
        self.b1 = b1
        
        # Valores estimados
        self.y_hat = self.b0 + self.b1 * self.x
        # Residuales
        self.residuos = self.y - self.y_hat
        # Varianza sigma2
        self.sigma2_hat = np.sum( self.residuos ** 2 ) / (self.n - 2)
        self.sigma_hat = np.sqrt(self.sigma2_hat)
        
        
    def varianza_estimadores(self):
        # Var(B0) = (1/n + xbarra^2 /Sxx)sigma2
        # Var(B1) = sigma2 / Sxx
        self.var_b0 = (1 / self.n + self.x_bar ** 2 / self.Sxx) * self.sigma2_hat
        self.var_b1 = self.sigma2_hat / self.Sxx

    def intervalo_confianza(self, alpha=0.05, focus=None):
        # IC para B0 y B1
        t_alpha = t.ppf(1-alpha/2, df=self.n - 2 )
        
        ic_b0 = [self.b0 - t_alpha * np.sqrt(self.var_b0),
                 self.b0 + t_alpha * np.sqrt(self.var_b0)]
        ic_b1 = [self.b1 - t_alpha * np.sqrt(self.var_b1),
                 self.b1 + t_alpha * np.sqrt(self.var_b1)]
        
        
        # Intervalo  para la varianza del modelo
        chi_lower = chi2.ppf(1-alpha/2, df= self.n-2)
        chi_upper = chi2.ppf(alpha/2, df= self.n-2)
        
        ic_sigma2 = [ (self.n-2) * self.sigma2_hat / chi_lower,
                     (self.n-2) * self.sigma2_hat / chi_upper]
        
        ic_b0 = [float(aux) for aux in ic_b0]
        ic_b1 = [float(aux) for aux in ic_b1]
        ic_sigma2 = [float(aux) for aux in ic_sigma2]
        
        
        
        answer = { 'b0': ic_b0, 'b1': ic_b1, 'betas': [ic_b0, ic_b1], 
                  'ic_sigma2': ic_sigma2}
        
        return answer.get(focus,  (ic_b0, ic_b1,ic_sigma2))
    
    
    def predict(self, x):
        """ Genera valores predichos
        y_hat = B0 + B1 * x
        """
        
        return self.b0 + self.b1 * x
        
    
    
    def intervalo_valor_medio(self, x0, alpha = 0.05):
        t_alpha = t.ppf(1-alpha/2, df=self.n - 2 )
        y0_hat = self.predict(x0)

        # Var(y0) = sigma2 * ( 1/n + (x'-x_barr)**2 / Sxx  )
        var_y0_hat = np.abs(self.sigma2_hat * ( 1 / self.n + (x0-self.x_bar) ** 2 / self.Sxx )) 

        
        ic_lower = float(y0_hat - t_alpha * np.sqrt(var_y0_hat))
        ic_upper = float(y0_hat + t_alpha * np.sqrt(var_y0_hat))
        
        return [ic_lower, ic_upper]
    
    

    
    def intervalo_prediccion(self, x0, alpha = 0.05):
        t_alpha = t.ppf(1-alpha/2, df=self.n - 2 )
        
        y0_hat = self.predict(x0)
        
        
        # Var(y0) = sigma2 * ( 1 + 1/n + (x'-x_barr)**2 / Sxx  )
        var_y0_hat = self.sigma2_hat * (1 + 1 / self.n + (x0-self.x_bar) ** 2 / self.Sxx ) 
        
        ic_lower = float(y0_hat - t_alpha * np.sqrt(var_y0_hat))
        ic_upper = float(y0_hat + t_alpha * np.sqrt(var_y0_hat))
        
        return [ic_lower, ic_upper]
        
        
        
    
            
        
        
    def prueba_hipotesis_b0(self, alpha=0.05, b0_0=0):
        # Prueba de hipótesis para H0: B0 = b0_0
        region_critica_izquierda = t.ppf(alpha/2, df=self.n - 2)
        region_critica_derecha = t.ppf(1 - alpha/2, df=self.n - 2)
        
        # Escribe aqui el código para la prueba de hipótesis
        # Regresa True si se rechaza H0 y False en caso contrario
        decision = False # Implementa la lógica de la prueba aquí

        return bool(decision)

    
    def prueba_hipotesis_b1(self, alpha=0.05, b1_0=0):
        # Prueba de hipótesis para H0: B1 = b1_0
        region_critica_izquierda = t.ppf(alpha/2, df=self.n - 2)
        region_critica_derecha = t.ppf(1 - alpha/2, df=self.n - 2)
        
        # Escribe aqui el código para la prueba de hipótesis
        # Regresa True si se rechaza H0 y False en caso contrario
        decision = False # Implementa la lógica de la prueba aquí

        return bool(decision)





""" Ejercicio.
Un ingeniero de pista quiere modelar el tiempo de vuelta (en segundos) de un coche de Fórmula
en función del porcentaje de desgaste de llantas. Se simulan 18 observaciones. Para ello, se
recogen los datos de desgaste y tiempo de vuelta en los arrays siguientes.
"""

# Desgaste en porcentaje (%)
desgaste = np.array([8, 12, 15, 18, 22, 25, 28, 32, 35,
                     41, 45, 52, 58, 63, 70, 77, 85, 92], dtype=float)

# Tiempo de vuelta en segundos (s)
tiempo_vuelta = np.array([77.73802849, 78.07704142, 79.08861312, 80.15381791, 79.81950798,
       80.35951783, 81.98752769, 82.22046084, 82.01831537, 83.70553603,
       83.82194938, 85.08056215, 86.58517736, 86.19203185, 87.5650493 ,
       89.52262748, 90.69230133, 92.7485484 ])

""" 
Con base en los datos anteriores, el ingeniero quiere ver si existe una relación lineal
entre el desgaste y el tiempo de vuelta, y en caso afirmativo, usar el modelo para
predecir el tiempo de vuelta en función del desgaste. Para ello, responde las siguientes
preguntas:
"""

# Llamamos a nuestra clase para ajustar un modelo de regresión lineal simple
modelo_f1 = RegresionLinealSimple(None, None)  # Añade los parámetros correspondientes

# ¿Cuál es el estimador de B0 y B1?
B0_f1 = None
print("El valor estimado para B0 (intersección con el eje y) es:", B0_f1)

B1_f1 = None 
print("El valor estimado para B1 (pendiente de la recta) es:", B1_f1)

# ¿Cuál es la media de desgaste y cuál es la media de tiempo de vuelta?
desgaste_media = None  
print("La media del porcentaje de desgaste es:", desgaste_media)

tiempo_media = None
print("La media del tiempo de vuelta (s) es:", tiempo_media)


"""
Si el coche tiene 15% de desgaste y 99% de desgaste,
¿cuál será el tiempo de vuelta esperado en cada caso?
"""
tiempo_15 = None
print("El tiempo de vuelta esperado con 15% de desgaste es (s):", tiempo_15)

tiempo_99 = None
print("El tiempo de vuelta esperado con 99% de desgaste es (s):", tiempo_99)


"""
Calcula el IC del valor medio y el IC de predicción para 15% y 99% de desgaste.
Compara y explica, en 2-3 líneas, por qué el intervalo de predicción es más amplio.
"""
ic_vm_15 = None # Intervalo de valor medio  para desgaste del 15%
ic_pr_15 = None  # Intervalo de predicción  para desgaste del 15%

ic_vm_99 = None # Intervalo de valor medio   para desgaste del 99%
ic_pr_99 = None # Intervalo de predicción  para desgaste del 99%

print("IC valor medio (95%) en 15%:", ic_vm_15)
print("IC predicción (95%) en 15%:", ic_pr_15)
print("IC valor medio (95%) en 99%:", ic_vm_99)
print("IC predicción (95%) en 99%:", ic_pr_99)


"""
Obtén los IC al 95% de B0, B1 y de la varianza sigma^2 del modelo.
(Recuerda llamar antes a varianza_estimadores()).
"""
modelo_f1.varianza_estimadores()
ics = None 
print("ICs al 95% (B0, B1, sigma^2):", ics)

"""
Usa las funciones de prueba (que agregaste a la clase) para contrastar:
H0: B0 = 0  y  H0: B1 = 0  con α = 0.05
Nota: tus métodos usan var_b0 y var_b1, asegúrate de haber llamado varianza_estimadores().
"""
decision_b0 = None
decision_b1 = None

print("¿Rechazamos H0: B0 = 0 al 5%? ->", decision_b0)
print("¿Rechazamos H0: B1 = 0 al 5%? ->", decision_b1)